In [1]:
### Importing required packages 
import os
from pprint import pprint
from dotenv import load_dotenv
import tweepy
import pandas as pd
import sqlite3
import re
import numpy as np

In [2]:
### Loading in the environment variables
load_dotenv()

### For Twitter API v2 
TWITTER_BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN", default = "OOPS")

In [3]:
### Establishing the client 

client = tweepy.Client(bearer_token = TWITTER_BEARER_TOKEN)

In [14]:
MAX_RESULTS = 100
query = '#COP26 lang:en'
response = client.search_recent_tweets(query = query, expansions=['author_id','attachments.media_keys','referenced_tweets.id','geo.place_id'], tweet_fields=['created_at','entities','context_annotations'], media_fields=['url','preview_image_url'], user_fields=['created_at','verified'], max_results = MAX_RESULTS)


In [ ]:
def lookup_tweet(ref_id) :
    response = client.get_tweets(ref_id, tweet_fields = ['entities'])
    for tweet in response.data :
        if hasattr(tweet, 'entities') and tweet['entities'] :
            url_entities = tweet.entities.get('urls') or [] 
        else :
            url_entities = []
        
    return url_entities    

In [37]:
url_records = []

for tweet in response.data :
    url_entities = tweet.entities.get('urls') or [] 
    
    if hasattr(tweet, "referenced_tweets") and tweet["referenced_tweets"]:
            referenced_tweets = tweet["referenced_tweets"]
            for ref in referenced_tweets:
                ref_id = [ref.id]
                url_entities = lookup_tweet(ref_id)
                urls = [{'status_id' : tweet.id, 'url' : ent['expanded_url']} for ent in url_entities]
                url_records += urls

    else :
        urls = [{'status_id' : tweet.id, 'url' : ent['expanded_url']} for ent in url_entities]
        url_records += urls
    
    
    

In [39]:
len(url_records)

121